<a href="https://colab.research.google.com/github/smitdesai1010/COVID-19-Detection/blob/master/COVID_19_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

files.upload()

In [ ]:
import json

token = {"username":"smitdesai1010","key":"41f928f2f4a0f0b462ed2ec45f383c82"}
with open('/content/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/kaggle.json ~/.kaggle/kaggle.json

!kaggle config set -n path -v /content
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset


import zipfile

zf = zipfile.ZipFile('/content/datasets/amanullahasraf/covid19-pneumonia-normal-chest-xray-pa-dataset/covid19-pneumonia-normal-chest-xray-pa-dataset.zip') 
zf.extractall()

!rm -rf 'datasets' #Deleting the the downloaded Zip files after extracting it 

In [44]:
import tensorflow as tf
import cv2
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [50]:
def create_dataset(name):
  org_path = '/content/'+ name
  train_path = '/content/train/'+ name
  val_path = '/content/validation/'+ name
  test_path = '/content/test/'+ name

  if not os.path.exists(train_path):
    os.mkdir(train_path)
    os.mkdir(val_path)
    os.mkdir(test_path)

  path, dirs, files = next(os.walk(org_path))
  file_count = len(files)
  i = 0  
    
  for file in os.listdir(org_path):
    
    if not os.path.isfile(os.path.join(org_path,file)):
      continue

    if i < file_count*0.75:
     shutil.copy2(os.path.join(org_path,file), os.path.join(train_path,file))
    
    elif i > file_count*0.9:
     shutil.copy2(os.path.join(org_path,file), os.path.join(test_path,file))

    else:
     shutil.copy2(os.path.join(org_path,file), os.path.join(val_path,file))
     
    i = i + 1 

https://medium.com/@mrgarg.rajat/training-on-large-datasets-that-dont-fit-in-memory-in-keras-60a974785d71

In [54]:
if not os.path.exists('/content/train'):
  !mkdir /content/train
  !mkdir /content/validation
  !mkdir /content/test
 
create_dataset('covid')
create_dataset('normal')
create_dataset('pneumonia')

!rm -rf 'covid'
!rm -rf 'normal'
!rm -rf 'pneumonia'